In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_log_error
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z


In [ ]:
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip


In [ ]:

train = pd.read_table('train.tsv')
test = pd.read_table('test_stg2.tsv')
print(train.shape, test.shape)

In [ ]:
train.head()

In [ ]:
train.info

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
test.describe()

In [ ]:
train.describe

In [ ]:
print(pd.isnull(train).sum())


In [ ]:

print(pd.isnull(test).sum())

In [ ]:


train["item_description"].fillna("No description yet", inplace=True)
train["brand_name"].fillna("None", inplace=True)
train["category_name"].fillna("Other", inplace=True)

In [ ]:
test["item_description"].fillna("No description yet", inplace=True)
test["brand_name"].fillna("None", inplace=True)
test["category_name"].fillna("Other", inplace=True)

In [ ]:
train.brand_name.fillna(value = "NoBrand", inplace = True)
test.brand_name.fillna(value = "NoBrand", inplace = True)

train.category_name.fillna(value = "Other/Other/Other", inplace = True)
test.category_name.fillna(value = "Other/Other/Other", inplace = True)

train.item_description.fillna(value = "No description yet", inplace = True)
test.item_description.fillna(value = "No description yet", inplace = True)

In [ ]:
print(pd.isnull(train).sum())

In [ ]:
print(pd.isnull(test).sum())

In [ ]:
train['general_category']='' 
train['subcategory_1'] = '' 
train['subcategory_2'] = ''

test['general_category']='' 
test['subcategory_1'] = '' 
test['subcategory_2'] = ''



In [ ]:
def transform_category_name(category_name):
    try:
        sub1, sub2, sub3 = category_name.split('/')
        return sub1, sub2, sub3
    except:
        return "none", "none", "none"

train['general_category'], train['subcategory_1'], train['subcategory_2'] = zip(*train['category_name'].apply(transform_category_name))
test['general_category'], test['subcategory_1'], test['subcategory_2'] = zip(*test['category_name'].apply(transform_category_name))
train.head(10)

In [ ]:
train.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
train['item_condition_id'].value_counts()
train['item_condition_id'].value_counts().plot(kind='bar')

In [ ]:
train['brand_name'].value_counts()
train['brand_name'].value_counts().plot(kind='bar')

In [ ]:
train['shipping'].value_counts()
train['shipping'].value_counts().plot(kind='bar')

In [ ]:
train['general_category'].value_counts().plot(kind='bar')
train['general_category'].value_counts()

In [ ]:
train['subcategory_1'].value_counts().plot(kind='pie')
train['subcategory_1'].value_counts()

In [ ]:
train['subcategory_1'].value_counts().plot(kind='bar')

In [ ]:

train['subcategory_2'].value_counts()
train['subcategory_2'].value_counts().plot(kind='pie')

In [ ]:
train['subcategory_2'][:300].value_counts().plot(kind='bar')

In [ ]:


scaler = MinMaxScaler()
X_train_item_condition_id = scaler.fit_transform(train['item_condition_id'].values.reshape(-1,1))

In [ ]:

lb_brand_name = LabelBinarizer(sparse_output=True)
X_train_brand = lb_brand_name.fit_transform(train['brand_name'])

lb_shipping = LabelBinarizer(sparse_output=True)
X_train_shipping = lb_shipping.fit_transform(train['shipping'])

lb_cat_0 = LabelBinarizer(sparse_output=True)
X_train_general_category = lb_cat_0.fit_transform(train['general_category'])

lb_cat_1 = LabelBinarizer(sparse_output=True)
X_train_subcategory_1 = lb_cat_1.fit_transform(train['subcategory_1'])

lb_cat_2 = LabelBinarizer(sparse_output=True)
X_train_subcategory_2 = lb_cat_2.fit_transform(train['subcategory_2'])

In [ ]:
tfidf_desc = TfidfVectorizer(max_features=50000, ngram_range=(1, 1), stop_words='english', norm='l2',lowercase=True)
name_vectorizer = CountVectorizer(stop_words='english')

X_train_descp = tfidf_desc.fit_transform(train['item_description'])
X_train_name = name_vectorizer.fit_transform(train['name'])

In [ ]:
from scipy.sparse import hstack

X_train = hstack((X_train_name, X_train_descp, X_train_brand, X_train_item_condition_id, X_train_shipping, X_train_general_category, X_train_subcategory_1, X_train_subcategory_2)).tocsr()

In [ ]:
X_train

In [ ]:
corr = train.corr()
corr


In [ ]:
import seaborn as sns

sns.heatmap(corr, xticklabels=corr.columns.values,yticklabels=corr.columns.values)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
for train_index, test_index in split.split(train, train['general_category']):
    train_idx = train_index
    test_idx = test_index

In [ ]:
log_prices = np.log1p(train.price)
test_data = X_train[test_idx]
log_test_prices = log_prices.iloc[test_idx]
train_data = X_train[train_idx]
log_train_prices = log_prices.iloc[train_idx]

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def performance(ytrue, ypredicted):
    print("MSE: ",np.sqrt(mean_squared_error(ytrue, ypredicted))) 
    print("MAE: ", np.sqrt(mean_absolute_error(ytrue, ypredicted))) 

In [ ]:
from sklearn.linear_model import SGDRegressor

# SkLearn SGD classifier
clf_ = SGDRegressor()
clf_.fit(train_data, log_train_prices)
performance(log_test_prices, clf_.predict(test_data))

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV

In [ ]:
ridge_lin_reg = Ridge(alpha=4, max_iter= 500)
ridge_lin_reg.fit(train_data, log_train_prices)

performance(log_test_prices,ridge_lin_reg.predict(test_data))

In [ ]:
from sklearn.svm import SVR

kernel_SVR = SVR(kernel='rbf', verbose=True, max_iter= 1000)
kernel_SVR.fit(train_data, log_train_prices)

performance(log_test_prices,kernel_SVR.predict(test_data))

In [ ]:
from lightgbm import LGBMRegressor
lgbm_model = LGBMRegressor(verbose=0, max_depth=15, n_estimators=30)
lgbm_model.fit(train_data, log_train_prices, verbose=0)

performance(log_test_prices,lgbm_model.predict(test_data))